In [15]:
import cv2
import numpy as np
from keras.models import model_from_json
import tkinter as tk
import tensorflow as tf
from tkinter import filedialog
from tkinter import *
from tkinter import Label, Tk

In [2]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [3]:
# Load model architecture from JSON file
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [4]:
# Load model weights from H5 file
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights("model/emotion_model.h5")
print("Loaded model from disk")



Loaded model from disk


In [5]:
# load weights into new model
emotion_model.load_weights("model/emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
def start_detection(source):
    if source == "webcam":
        cap = cv2.VideoCapture(0)
    else:
        video_path = filedialog.askopenfilename()
        if video_path:
            cap = cv2.VideoCapture(video_path)
        else:
            return

    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            break

        frame = cv2.resize(frame, (1280, 720))
        face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

            emotion_prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(emotion_prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        #press "s" to Quit
        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [7]:
def start_webcam_detection():
    start_detection("webcam")

def start_video_detection():
    start_detection("video")

In [16]:
# Create GUI window
root = tk.Tk()
root.title("Emotion Detection")
root.geometry('800x600')
root.configure(background='#CDCDCD')


# Add buttons for webcam and video detection
label1 = Label(root, background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(root)
label1.pack(side='bottom', expand='True')
heading = Label(root,text='Emotion Detection',pady=20,font=('arial',25,'bold'))
heading.configure(background='#CDCDCD',foreground="#364156")
heading.pack()

webcam_button = tk.Button(root, text="Use Webcam", command=start_webcam_detection)
webcam_button.pack(pady=10)

video_button = tk.Button(root, text="Use Video", command=start_video_detection)
video_button.pack(pady=10)

root.mainloop()
#press "s" to Quit